In [3]:
import orthoslicer as ort
from scipy.ndimage import gaussian_filter
import matplotlib.pyplot as plt
import numpy as np
import os
import h5py
import image_creation as ic

def crop_image(dirpath, imagefile, create_crop_image=False, load_crop_image=False, just_plot=False, also_plot=False):
	map_joiner = lambda path: os.path.join(dirpath, path)

	if create_crop_image and load_crop_image:
		raise RuntimeError('Can not both load and create crop images')

	if create_crop_image:
		print('Loading images')
		with h5py.File(map_joiner(imagefile), "r") as f:
			img = f['corrected_img'][()]
			cd = f['cd'][()]
			smaps = f['smaps'][()]
			
		nlen = 72
		nx = 70
		ny = 70
		nz = 130 #45

		#nx = 60
		#ny = 60
		#nz = 70

		crop_box = [(nx,nx+nlen),(ny,ny+nlen),(nz,nz+nlen)]
		print('Cropping')
		new_img = ic.crop_5d_3d(img, crop_box)
		new_cd = ic.crop_4d_3d(cd, crop_box).astype(np.float32)
		new_smaps = ic.crop_4d_3d(smaps, crop_box)

		new_cd = gaussian_filter(new_cd, sigma=0.8)

		vessel_mask = new_cd > np.quantile(new_cd, 0.995)
		vessel_mask = np.all(vessel_mask, axis=0)

		if just_plot or also_plot:
			ort.image_nd(new_img)
			ort.image_nd(new_cd)
			#ort.image_nd(new_cd + vessel_mask[None,...]*50.0)
			ort.image_nd(vessel_mask.astype(np.float32), max_clim=True)

			if just_plot:
				return (None, None)



	return img, smaps